T7. COLUMNS RECONSTRUCTION

    T7.1 RECTANGULAR COLUMNS 

In this script, we reconstruct columns from the instance segmentation and reference heights.
We need:
 - T0 for classes and objects
 - T1: objest and objects_ids
 - T5: reference levels

## LIBRARIES

In [ ]:
#IMPORT PACKAGES
from rdflib import Graph, URIRef
import os.path
import importlib
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import multiprocessing
import random as rd
import pandas as pd
# from tabulate import tabulate
import cv2
import laspy
import json
from scipy.spatial.transform import Rotation   
import copy
import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

# #import utils
# from context import utils
# import utils as utl
# import utils.t1_utils as t1


In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

INPUTS

In [ ]:
name='steel_labels'

path=Path(os.getcwd()).parents[2]/'data'
pcd_input_path=os.path.join(path,f'{name}.las')
class_file=path/'_classes.json'

OUTPUTS

In [ ]:
name=name.split('_')[0]
json_output_path=os.path.join(path,f'{name}_columns.json') 
geometry_output_path= os.path.join(path,f'{name}_columns.obj') # these are the bounding surfaces of the reference levels (optional)

IMPORT CLASSES

In [ ]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

THRESHOLDS

In [ ]:
t_level=0.5
t_distance=0.5
t_thickness=0.15
t_orthogonal=0.5
t_topo_floors_ceilings=False
t_intersection=0.5
# t_topology=5



NODES

In [ ]:
print(pcd_input_path)

laz=laspy.read(pcd_input_path)

In [ ]:
laz=laspy.read(pcd_input_path)

In [ ]:
pcdNodes=[]

#split pcd per object
for i in np.unique(laz['classes']):
    idx=np.where(laz['classes']==i)
    points=laz.xyz[idx]
    # colors=np.array([laz.red[idx],laz.green[idx],laz.blue[idx]])
    object_labels=laz['objects'][idx]

    class_obj=next((class_obj for class_obj in json_data['classes'] if float(class_obj['id']) ==i), json_data['classes'][0])
    class_name=class_obj['name']

    # pcd.colors=o3d.utility.Vector3dVector(colors)
    for j in np.unique(object_labels):
        
        new_points=points[np.where(object_labels==j)]
        if new_points.shape[0]>100:
            pcd=o3d.geometry.PointCloud()
            pcd.points=o3d.utility.Vector3dVector(new_points)

            pcdNodes.append(PointCloudNode(resource=pcd,
                                        class_id=i,
                                        object_id=j,
                                        color=ut.random_color(),
                                            name=class_name+f'_{j}'))

print(f'{len(pcdNodes)} pcdNodes created!')

GRAPH_IMPORT GRAPH

In [ ]:
graphPath = str(path / f'{name}Graph.ttl')
graph = Graph().parse(graphPath, format="turtle")

print(graph)

graph=Graph().parse(graphPath)
print(graph)
nodes=tl.graph_to_nodes(graph)
print(nodes)
columnNodes=[n for n in nodes if 'Columns' in n.subject and type(n)==PointCloudNode]
wallNodes=[n for n in nodes if 'Walls' in n.subject and type(n)==PointCloudNode]
ceilingsNodes=[n for n in nodes if 'Ceilings' in n.subject and type(n)==PointCloudNode]
floorsNodes=[n for n in nodes if 'Floors' in n.subject and type(n)==PointCloudNode]
print(f'{len(columnNodes)} columnNodes detected!')
print(f'{len(wallNodes)} wallNodes detected!')
print(f'{len(ceilingsNodes)} ceilingsNodes detected!')
print(f'{len(floorsNodes)} floorsNodes detected!')

IMPORT PCD- MATCHING GRAPH

In [ ]:
for n in columnNodes:
    idx=np.where((laz['classes']==n.class_id) & (laz['objects']==n.object_id))
    pcd=o3d.geometry.PointCloud()
    pcd.points=o3d.utility.Vector3dVector(laz.xyz[idx])
    n.resource=pcd
    n.get_oriented_bounding_box()
    n.orientedBoundingBox.color=[1,0,0]

In [ ]:
joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(ut.literal_to_array(n.color)) for n in wallNodes if n.resource is not None])
# o3d.visualization.draw_geometries([joined_pcd])

IMPORT LEVELS

In [ ]:
levelNodes=[n for n in nodes if 'level' in n.subject]
referenceNodes=[]
for l in levelNodes:
    new_graph=ut.get_subject_graph(graph,levelNodes[0].subject)
    n=SessionNode(graph=new_graph)
    n.get_oriented_bounding_box()
    n.resource=o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(n.orientedBoundingBox)
    referenceNodes.append(n) 
levelNodes=referenceNodes
print(f'{len(levelNodes)} levelNodes detected!')

CEILINGS AND FLOORS

In [ ]:
for n in ceilingsNodes+floorsNodes: # this is quite slow because you iterate through 2 scalar fields every time
    idx=np.where((laz['classes']==n.class_id) & (laz['objects']==n.object_id))
    pcd=o3d.geometry.PointCloud()
    pcd.points=o3d.utility.Vector3dVector(laz.xyz[idx])
    n.resource=pcd
    n.get_oriented_bounding_box()
    n.orientedBoundingBox.color=[1,1,0]

PROCESSING

READ POINTS

A_BASE CONSTRAINT

In [ ]:
for n in columnNodes:
    #compute minheight of the resource at 0.1% of the height 
    z_values = np.sort(np.asarray(n.resource.points)[:,2])
    minheight = np.percentile(z_values, 0.1)

    #compute base constraint. select the intersecting level that is closest to the bottom of the wall. Else, just take first levelNode.
    nearby_ref_levels= tl.select_nodes_with_intersecting_bounding_box(n,levelNodes)
    n.base_constraint= next((n for n in nearby_ref_levels if np.absolute(n.height-minheight)<t_level),levelNodes[0])  if nearby_ref_levels else levelNodes[0] 
    
    #compute base offset
    n.base_offset=minheight-n.base_constraint.height
    print(f'name: {n.name}, base_constraint: {n.base_constraint.name}, base_offset: {n.base_offset}')


B_TOP CONSTRAINT

In [ ]:
points = np.sort(np.asarray(n.resource.points)[:,0:2])

In [ ]:
for n in columnNodes:
    #compute maxheight of the resource at 0.1% of the height (absolute minimum might be wrong)
    z_values = np.sort(np.asarray(n.resource.points)[:,2])
    minheight = np.percentile(z_values, 0.1)
    maxheight = np.percentile(z_values, 99.9)

    #compute base constraint. select the intersecting level that is closest to the top of the wall. Else, just take last levelNode.
    nearby_ref_levels= tl.select_nodes_with_intersecting_bounding_box(n,levelNodes)
    n.top_constraint= next((n for n in nearby_ref_levels if np.absolute(n.height-maxheight)<t_level),levelNodes[-1]) if nearby_ref_levels else levelNodes[-1] #this is a link!
    
    #compute base offset
    n.top_offset=maxheight-n.top_constraint.height

    #compute wall height
    n.height=maxheight-minheight
    print(f'name: {n.name}, top_constraint: {n.top_constraint.name}, top_offset: {n.top_offset}')


In [ ]:
joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(ut.literal_to_array(n.color)) for n in columnNodes if n.resource is not None])
for n in columnNodes:
    n.orientedBoundingBox.color=[1,0,0]
o3d.visualization.draw_geometries([joined_pcd]+[n.orientedBoundingBox for n in columnNodes])

COLUMN ORIENTATION

In [ ]:
for n in columnNodes:    
    # Main plane
    n.plane_model, inliers = n.resource.segment_plane(distance_threshold=0.01,
                                            ransac_n=3,
                                            num_iterations=1000)
    
    # Center at correct heigth   
    n.faceCenter=n.resource.select_by_index(inliers).get_center()  
    n.faceCenter[2]=n.base_constraint.height + n.base_offset

    #compute the normal of the plane in 2D (z=0)
    n.normal=n.plane_model[:3]
    n.normal[2]=0
    n.normal/=np.linalg.norm(n.normal)

    #compute the sign.
    #if n.orientedBoundingBox width is > than 0.1, the sign is  given by the dotproduct of the normal of the face with the vector between the center of the face and the center of the oriented bounding box
    boxCenter=n.orientedBoundingBox.get_center()
    boxCenter[2]=n.base_constraint.height + n.base_offset
    n.sign=np.sign(np.dot(n.normal,n.faceCenter-boxCenter))

    #if n.orientedBoundingBox width < t_thickness, take a look at the ceiling and floor nodes to see on which side they are, and use them to spawn the wall away from these nodes
    if n.orientedBoundingBox.extent[2]<=t_thickness:   
        #combine list
        combined_list = ceilingsNodes + floorsNodes
        #create reference pcd from these resources
        referencePcd,identityArray=gmu.create_identity_point_cloud([n.resource for n in combined_list if n.resource is not None])
        #find nearest point near the top and the bottom 
        topPoint=copy.deepcopy(boxCenter)
        topPoint[2]=n.base_constraint.height + n.base_offset+n.height
        bottomPoint=boxCenter
        idx,distances=gmu.compute_nearest_neighbors(np.asarray([topPoint,bottomPoint]),np.asarray(referencePcd.points)) 
        points=np.asarray(referencePcd.points)[idx[:,0]]
        #compute orthogonal distance to the plane and select node with lowest distance
        idx=idx[np.argmin(np.absolute(np.einsum('i,ji->j',n.normal,points-boxCenter))) ][0]
        index=identityArray[idx]
        referenceNode=combined_list[index]
        point=np.asarray(referencePcd.points)[idx]
        point[2]=n.base_constraint.height + n.base_offset
        n.sign=np.sign(np.dot(n.normal,point-boxCenter))
    
    #flip the normal if it points inwards
    n.normal*=-1 if n.sign==-1 else 1

    print(f'name: {n.name}, plane: {n.plane_model}, inliers: {len(inliers)}/{len(np.asarray(n.resource.points))}')
          


COMPUTE COLUMN THICKNESS 

In [ ]:
for n in columnNodes:
    #compute the normals of the wall
    pcd_tree = o3d.geometry.KDTreeFlann(n.resource)
    n.resource.estimate_normals() if not n.resource.has_normals() else None

    #for every 10th point in P, that has the same normal as the dominant plane, select nearest points in P that meet a distance threshold    
    points=np.asarray(n.resource.points)[::100]
    normals=np.asarray(n.resource.normals)[::100]
    idx=np.where(np.absolute(np.einsum('i,ji->j',n.normal,normals))>0.9)
    points=points[idx]
    normals=normals[idx]
    distances=[]

    for p,q in zip(points,normals):
        #compute distances
        [k, idx, _] = pcd_tree.search_radius_vector_3d(p, t_distance)        
        #retain only the distances for which the normal is within 0.7 radians of the normal of the point
        kNormals=np.asarray(n.resource.normals)[idx]
        indices=np.asarray(idx)[np.where(np.absolute(np.einsum('i,ji->j',q,kNormals))>0.9)]
        #compute the dotproduct between the point and the normals of the points in the radius
        vectors=p-np.asarray(n.resource.select_by_index(indices).points)
        #keep the max distance (orthogonal distance between the planes)
        distances.append(np.absolute(np.einsum('i,ji->j', q, vectors)).max())

    #take the distance at the 99% percentile
    distance = np.percentile(np.sort(np.array(distances)), 90) 
    n.columnThickness=distance if distance > t_thickness else t_thickness

    print(f'name: {n.name}, BB_extent: {n.orientedBoundingBox.extent}, columnThickness: {n.columnThickness}')


    ## ADD columnWidth: {}


In [ ]:
o3d.visualization.draw_geometries([n.resource for n in columnNodes if n.columnThickness <=t_thickness]+
                                  [n.orientedBoundingBox for n in columnNodes])

COMPUTE COLUMN AXIS

In [ ]:
for n in columnNodes:     
    
    #offset the center of the plane with half the column thickness in the direction of the normal of the plane  
    columnCenter=n.faceCenter-n.normal*n.columnThickness/2 

    columnCenter[2]=n.base_constraint.height + n.base_offset

    #project axis aligned bounding points on the plane
    box=n.resource.get_axis_aligned_bounding_box()    
    points=np.asarray(box.get_box_points())
    points[:,2]=n.base_constraint.height + n.base_offset

    #translate the points to the plane
    vectors=points-columnCenter
    translation=np.einsum('ij,j->i',vectors,n.normal)
    points=points - translation[:, np.newaxis] * n.normal

    # Calculate the pairwise distances between all boundary points
    distances = np.linalg.norm(points[:, np.newaxis] - points, axis=2)

    # Get the indices of the two points with the maximum distance
    max_indices = np.unravel_index(np.argmax(distances), distances.shape)

    # Retain only the two points with the maximum distance
    n.boundaryPoints = points[max_indices,:]

    #create the axis
    n.axis=o3d.geometry.LineSet(points=o3d.utility.Vector3dVector(n.boundaryPoints),lines=o3d.utility.Vector2iVector([[0,1]])).paint_uniform_color([0,0,1])
    n.startPoint=n.boundaryPoints[0]
    n.endPoint=n.boundaryPoints[1]
    # Calculate the length
    n.columnWidth = np.linalg.norm(n.boundaryPoints[0] - n.boundaryPoints[1])

    print(f'name: {n.name}, ColumnWidth: {n.columnWidth}')


In [ ]:
o3d.visualization.draw_geometries([joined_pcd]+
                                  [n.orientedBoundingBox for n in columnNodes]+
                                  [n.axis for n in columnNodes]+
                                  [o3d.geometry.PointCloud(o3d.utility.Vector3dVector(n.boundaryPoints)) for n in columnNodes])

COMPUTE COLUMN GEOMETRY

In [ ]:
for n in columnNodes:
    pointList=[]
    points=np.asarray(n.axis.points)
    # pointList.extend(points+n.sign*n.normal*n.wallThickness/2)
    pointList.extend(points+n.normal*n.columnThickness/2)

    # pointList.extend(points-n.sign*n.normal*n.wallThickness/2)
    pointList.extend(points-n.normal*n.columnThickness/2)

    pointList.extend(np.array(pointList)+np.array([0,0,n.height]))
    pcd=o3d.geometry.PointCloud(points=o3d.utility.Vector3dVector(pointList))

    box=pcd.get_oriented_bounding_box()
    n.column=o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(box)
    n.column.paint_uniform_color(ut.literal_to_array(n.color))
    n.columnBox=o3d.geometry.LineSet.create_from_oriented_bounding_box(box)
    n.columnBox.paint_uniform_color([0,0,1])

    print(f'name: {n.name}, Column: {n.column}')


In [ ]:
o3d.visualization.draw_geometries([joined_pcd]+
                                  [n.columnBox for n in columnNodes]+
                                  [n.axis for n in columnNodes]+
                                  [o3d.geometry.PointCloud(o3d.utility.Vector3dVector(n.boundaryPoints)) for n in columnNodes])

Compute Wall topology

In [ ]:
def intersect_line_2d(p0, p1, q0, q1,strict=True):
    """
    Compute the intersection between two lines in 3D.
    Each line is defined by a pair of points.
    
    Parameters:
    - p0, p1: points (arrays) defining the first line.
    - q0, q1: points (arrays) defining the second line.
    - strict: if True, the intersection point must lie within the line segments.
    
    Returns:
    - The intersection point as a numpy array if it exists, otherwise None.
    """
    # Direction vectors of the lines
    dp = p1 - p0
    dq = q1 - q0
    
    # Matrix and vector for the linear system
    A = np.vstack((dp, -dq)).T
    b = q0 - p0
    
    # Solve the linear system
    try:
        t, u = np.linalg.solve(A, b)
    except np.linalg.LinAlgError:
        # The system is singular: lines are parallel or identical
        return None
    
    # Intersection point
    intersection = p0 + t * dp
    
    if strict:
    # Since the system has a solution, check if it lies within the line segments
        if np.allclose(intersection, q0 + u * dq):
            return intersection
        else:
            return None
    else:
        return intersection


BIM NODES FOR COLUMNS

In [ ]:
columnNodesBIM = []
for n in columnNodes:
    b = BIMNode(
        derivedFrom=n.subject,  # this should be a URI
        resource=n.column,
        base_constraint=n.base_constraint.subject,
        base_constraint_name=n.base_constraint.name,
        base_offset=n.base_offset,
        top_constraint=n.top_constraint.subject,
        top_constraint_name=n.top_constraint.name,
        top_offset=n.top_offset,
        height=n.height,
        columnsThickness=n.columnThickness, 
        columnsLength=n.columnWidth,  
        normal=n.normal,
        startpoint=n.boundaryPoints[0],
        endpoint=n.boundaryPoints[1],
        color=n.color
    )
    columnNodesBIM.append(b)

new_graph = tl.nodes_to_graph(columnNodesBIM, overwrite=True)

# Remove all BIM nodes from the original graph
for n in columnNodesBIM:
    graph.remove((URIRef(n.subject), None, None))

# Add new BIM nodes to the graph
graph = graph + new_graph

# Serialize the graph to a file
print(graph.serialize(graphPath, format="turtle"))

In [ ]:
from scipy.spatial import distance

for n in columnNodesBIM:
    n.orthogonalStartpoint = n.startpoint + n.normal * n.columnsThickness / 2
    n.orthogonalEndpoint = n.endpoint + n.normal * n.columnsThickness / 2

for n in columnNodesBIM:
    axisPoints = np.array([n.startpoint, n.endpoint])

    referencePcd, identityArray = gmu.create_identity_point_cloud([w.resource for w in columnNodesBIM if w != n])
    reference_points = np.asarray(referencePcd.points)

    idx, distances = gmu.compute_nearest_neighbors(axisPoints, reference_points, n=10)

    idx = idx[np.where(distances < t_intersection)]
    points = np.asarray(referencePcd.points)[idx]
    nearbyColumns = [columnNodesBIM[i] for i in identityArray[idx]]

    print(f'found {len(nearbyColumns)} nearby columns for {n.name}')

    for w in nearbyColumns:
        intersection_point = intersect_line_2d(n.startpoint, n.endpoint, w.startpoint, w.endpoint, strict=False)
        if intersection_point is not None:
            d = distance.euclidean(intersection_point, n.startpoint)
            if d < t_intersection:
                print(f'Intersection between {n.name} and {w.name} at {intersection_point}, distance: {d}')

    for w in nearbyColumns:
        intersection_point = intersect_line_2d(n.orthogonalStartpoint, n.orthogonalEndpoint, w.startpoint, w.endpoint)
        if intersection_point is not None:
            d = distance.euclidean(intersection_point, n.orthogonalStartpoint)
            if d < t_orthogonal:
                print(f'Orthogonal intersection between {n.name} and {w.name} at {intersection_point}, distance: {d}')

    break  # You may remove this break statement if you want to iterate over all column nodes


EXPORT

COMPUTE THE CENTRE OF THE COLUMN

In [ ]:
# Compute the center of the column for each object
for obj in json_data["objects"]:
    startpoint = np.array([obj["startpoint"]["x"], obj["startpoint"]["y"], obj["startpoint"]["z"]])
    endpoint = np.array([obj["endpoint"]["x"], obj["endpoint"]["y"], obj["endpoint"]["z"]])

    # Compute the center as the average of start and end points
    center = (startpoint + endpoint) / 2.0

    # Update the object with the center coordinates
    obj["center"] = {
        "x": center[0],
        "y": center[1],
        "z": center[2]
    }

print(center)

json with metadata

In [ ]:
#declare json
json_data = {
        "filename": ut.get_filename(json_output_path),
        "objects": []
    }
#fill json
for n in columnNodes:
    obj = {
            "name": n.name,
            "base_constraint":n.base_constraint.name,
            "base_offset":n.base_offset,
            "top_constraint":n.top_constraint.name,
            "top_offset":n.top_offset,
            "height": n.height,
            "columnThickness": n.columnThickness,
            "columnWidth": n.columnWidth,
            "normal": {
                "x": n.normal[0],
                "y": n.normal[1],
                "z": n.normal[2]
            },
            "startpoint": {
                "x": n.boundaryPoints[0][0],
                "y": n.boundaryPoints[0][1],
                "z": n.boundaryPoints[0][2]
            }
            ,
            "endpoint": {
                "x": n.boundaryPoints[1][0],
                "y": n.boundaryPoints[1][1],
                "z": n.boundaryPoints[1][2]
            }
            }
    json_data["objects"].append(obj)


WRITE THE JSON

In [ ]:
#write this information to the 3D detection json
with open(json_output_path, "w") as json_file:
    json.dump(json_data, json_file, indent=4)
print("JSON data written to file:", json_output_path)

OBJ WITH COLUMNS

In [ ]:
def write_obj_with_submeshes(filename, meshes, mesh_names):
    """
    Write multiple Open3D TriangleMesh objects to a single OBJ file with submeshes.

    Parameters:
    - filename: str, the name of the output OBJ file.
    - meshes: list of open3d.geometry.TriangleMesh, the meshes to write.
    - mesh_names: list of str, the names of the submeshes.
    """
    if len(meshes) != len(mesh_names):
        raise ValueError("meshes and mesh_names must have the same length")

    vertex_offset = 1  # OBJ files are 1-indexed
    with open(filename, 'w') as file:
        for mesh, name in zip(meshes, mesh_names):
            file.write(f"g {name}\n")  # Start a new group for the submesh

            # Write vertices
            for vertex in mesh.vertices:
                file.write(f"v {vertex[0]} {vertex[1]} {vertex[2]}\n")

            # Write faces, adjusting indices based on the current offset
            for triangle in mesh.triangles:
                adjusted_triangle = triangle + vertex_offset
                file.write(f"f {adjusted_triangle[0]} {adjusted_triangle[1]} {adjusted_triangle[2]}\n")

            # Update the vertex offset for the next mesh
            vertex_offset += len(mesh.vertices)

# Example usage:
# Assuming mesh1 and mesh2 are your Open3D TriangleMesh objects
mesh1 = o3d.geometry.TriangleMesh.create_sphere(radius=1.0)
mesh1.compute_vertex_normals()

mesh2 = o3d.geometry.TriangleMesh.create_box(width=1.0, height=1.0, depth=1.0)
mesh2.compute_vertex_normals()

write_obj_with_submeshes(geometry_output_path, [n.column for n in columnNodes], [n.name for n in columnNodes])

graph with metadata

In [ ]:
columnNodesBIM=[]
for n in columnNodes:
    b=BIMNode(subject=n.subject+'_BIM',
            derivedFrom=n.subject, #this should be a URI
            resource=n.column,
            base_constraint=n.base_constraint.subject,
            base_constraint_name=n.base_constraint.name,
            base_offset=n.base_offset,
            top_constraint=n.top_constraint.subject,
            top_constraint_name=n.top_constraint.name,
            top_offset=n.top_offset,
            height=n.height,
            columnThickness=n.columnThickness,
            columnWidth=n.columnWidth,
            normal=n.normal,
            startpoint=n.boundaryPoints[0],
            endpoint=n.boundaryPoints[1],
            color=n.color)
    columnNodesBIM.append(b)
new_graph=tl.nodes_to_graph(columnNodesBIM,overwrite=True)

#remove all BIM nodes from original graph
for n in columnNodesBIM:
    graph.remove((URIRef(n.subject),None,None))
graph=graph+new_graph
print(graph.serialize(graphPath, format="turtle"))